In [1]:
import os
from git import Repo
from pathlib import Path

import pandas as pd
import numpy as np
import secrets
import spacy
from spacy.util import load_config
from spacy.cli import apply
from spacy.scorer import Scorer, Example
from spacy.tokens import DocBin, Doc
import mlflow
from mlflow.entities import ViewType
import optuna

import scripts.sent_relevance.operations as ops
from scripts.utils.config import Config
from scripts.utils import flatten_config
from scripts.utils.spacy import load_spacy, train, load_metrics
from scripts.utils.optuna import ArchiveBestModelCallback

# Setup MLFLow

In [2]:
experiment_name = "sent_relevance_models"
task = "sent_relevance"

# Provide an Experiment description that will appear in the UI
experiment_description = (
    "Prototype model architectures for sentence relevance classifier."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "qjn",
    "task": task,
    "mlflow.note.content": experiment_description,
}

hyperparams = set([])

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment(experiment_name)
mlflow.set_experiment_tags(experiment_tags)

2025-03-20 13:41:12,083 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): 127.0.0.1:8080
2025-03-20 13:41:12,111 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=sent_relevance_models HTTP/1.1" 404 115
2025-03-20 13:41:12,117 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:41:12,143 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/experiments/create HTTP/1.1" 200 43
2025/03/20 13:41:12 INFO mlflow.tracking.fluent: Experiment with name 'sent_relevance_models' does not exist. Creating a new experiment.
2025-03-20 13:41:12,152 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:41:12,159 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "GET /api/2.0/mlflow/experiments/get?experiment_id=485071347047209202 HTTP/1.1" 200 296
2025-03-20 13:41:12,165 - urllib3.connectionpool - DEBUG - Resettin

In [4]:
def mlflow_log_eval(run_name, params, model_path, nested=False):
    
    # Evaluate model
    metrics = load_metrics(model_path)
    best_model_path = os.path.join(model_path, "model-best")

    # Load model params
    repo = Repo(config._LOCAL_PROJECT_DIR, search_parent_directories=True)
    params['git_hash'] = repo.heads.main.commit.hexsha

    # Reshape params for logging
    params = flatten_config(params)
    # XXX: This migth have a bug converting children to strings?
    params = {k.replace("@","_AT_"): v for k,v in params.items()}

    with mlflow.start_run(run_name=run_name, nested=nested) as run:
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)
        mlflow.spacy.log_model(load_spacy(best_model_path), run_name)

    return metrics

# Setup Experiments

In [5]:
config = Config()
train_path = config.get_data_path(f"{task}.article_text_train")
dev_path = config.get_data_path(f"{task}.article_text_dev")
base_cfg = config.get_file_path(f"{task}.base_cfg")
full_cfg = config.get_file_path(f"{task}.full_cfg")
out_path = config.get_file_path(f"{task}.trained_model")
out_path_scratch = config.get_file_path(f"{task}.trained_model", scratch=True)

# Quickstart Model

In [6]:
train(train_path, dev_path, full_cfg, out_path)

params = dict(load_config(full_cfg).interpolate())
mlflow_log_eval("quickstart_model", params, out_path)

ℹ Saving to output directory:
/Users/eric/Dev/quantify-news/models/sent_relevance
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-20 13:41:15,208] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       71.43    0.71
  0     200          39.32       60.44    0.60
  2     400          24.72       66.08    0.66
  3     600          16.28       73.09    0.73
  5     800          10.69       77.53    0.78
  7    1000           6.50       80.40    0.80
 10    1200           4.50       81.27    0.81
 13    1400           2.95       82.51    0.83
 17    1600           2.05       

2025-03-20 13:42:26,532 - git.util - DEBUG - Failed checking if running in CYGWIN due to: FileNotFoundError(2, 'No such file or directory')
2025-03-20 13:42:26,538 - git.cmd - DEBUG - Popen(['git', 'cat-file', '--batch-check'], cwd=/Users/eric/Dev/quantify-news, stdin=<valid stream>, shell=False, universal_newlines=False)


Training time: 73.9s


2025-03-20 13:42:26,561 - git.cmd - DEBUG - Popen(['git', 'check-ignore', '/Users/eric/Dev/quantify-news/.venv/lib/python3.12/site-packages'], cwd=/Users/eric/Dev/quantify-news, stdin=None, shell=False, universal_newlines=False)
2025-03-20 13:42:26,588 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:42:26,609 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 962
2025-03-20 13:42:26,619 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:42:26,675 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025-03-20 13:42:26,685 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:42:26,712 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025/03/20 13:42:35 WARNING mlflow.models.model: Model logged without a s

🏃 View run quickstart_model at: http://127.0.0.1:8080/#/experiments/485071347047209202/runs/bba5128d3d054bf09b3267803d05f33e
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/485071347047209202


{'cats_micro_p': 0.8260869565,
 'cats_micro_r': 0.7169811321,
 'cats_micro_f': 0.7676767677,
 'cats_macro_p': 0.6993103448,
 'cats_macro_r': 0.4926436782,
 'cats_macro_f': 0.5532451819,
 'cats_f_per_type.WHO.p': 1.0,
 'cats_f_per_type.WHO.r': 0.4,
 'cats_f_per_type.WHO.f': 0.5714285714,
 'cats_f_per_type.WHAT.p': 0.6,
 'cats_f_per_type.WHAT.r': 0.6666666667,
 'cats_f_per_type.WHAT.f': 0.6315789474,
 'cats_f_per_type.WHERE.p': 1.0,
 'cats_f_per_type.WHERE.r': 0.5,
 'cats_f_per_type.WHERE.f': 0.6666666667,
 'cats_f_per_type.WHEN.p': 0.0,
 'cats_f_per_type.WHEN.r': 0.0,
 'cats_f_per_type.WHEN.f': 0.0,
 'cats_f_per_type.IRRELEVANT.p': 0.8965517241,
 'cats_f_per_type.IRRELEVANT.r': 0.8965517241,
 'cats_f_per_type.IRRELEVANT.f': 0.8965517241,
 'cats_score': 0.8350646821}

# Hyperparams

In [7]:
def objective_base(trial, overrides = {}):
    print("Training with overrides:\n", overrides)
    ops.train(base_cfg, full_cfg, train_path, dev_path, out_path_scratch, overrides)
    
    # Train will keep base config and apply overrides at run-time.
    # So we load the config with the overrides for logging.
    params = dict(load_config(full_cfg, overrides).interpolate())

    run_name = f"optuna_trial_{trial.number}"
    metrics = mlflow_log_eval(run_name, params, out_path_scratch, nested=True)

    return metrics['cats_macro_f']

In [8]:
study = optuna.create_study(study_name=experiment_tags['task'],
                            direction="maximize",
                            storage=config.get_param(f"{task}.optuna_db"),
                            load_if_exists=True)

[I 2025-03-20 13:42:37,372] Using an existing study with name 'sent_relevance' instead of creating a new one.


In [9]:
archiver = ArchiveBestModelCallback(out_path=out_path, out_path_scratch=out_path_scratch)

In [10]:
def get_best(): 
    best = mlflow.search_runs(
        experiment_names=[experiment_name],
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=1,
        order_by=["metrics.cats_macro_f DESC"],
    ).iloc[0].T
    return best

def best_metrics():
    best = get_best()
    return best[best.index.str.startswith("metrics")].to_dict()

def best_params(keys):
    best = get_best()
    return {key: best.loc["params." + key] for key in keys}
    
def best_model():
    best = get_best()
    return os.path.join(best['artifact_uri'], best['tags.mlflow.runName'], 'model.spacy')

## Start Size

In [11]:
hp = "training.batcher.size.start"
hyperparams.add(hp)

def objective(trial):
    hp_start_size = trial.suggest_int(hp, 1, 100)  # Tune batch start
    overrides = {hp: hp_start_size}
    return objective_base(trial, overrides)
    

In [12]:
# Note: as currently configured, the optuna_db goes into the caller notebook folder
with mlflow.start_run(run_name="opt_batch_start_size"):
    study.optimize(objective, n_trials=10, callbacks=[archiver])

2025-03-20 13:42:37,887 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:42:37,916 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 970


Training with overrides:
 {'training.batcher.size.start': 30}
✔ Auto-filled config with all values
✔ Saved config
/Users/eric/Dev/quantify-news/scripts/sent_relevance/spacy_config.cfg
You can now add your data and train your pipeline:
python -m spacy train spacy_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory:
/Users/eric/Dev/quantify-news/models/sent_relevance
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-20 13:42:42,405] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       71.43    0.71
  0     200          39.09       59.

2025-03-20 13:44:02,312 - git.cmd - DEBUG - Popen(['git', 'cat-file', '--batch-check'], cwd=/Users/eric/Dev/quantify-news, stdin=<valid stream>, shell=False, universal_newlines=False)


Training time: 82.3s


2025-03-20 13:44:02,338 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:44:02,361 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 1074
2025-03-20 13:44:02,371 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:44:02,442 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025-03-20 13:44:02,453 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:44:02,480 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025/03/20 13:44:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-20 13:44:09,692 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-0

...

🏃 View run optuna_trial_40 at: http://127.0.0.1:8080/#/experiments/485071347047209202/runs/c1095f3fdda14487988173bad7746741
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/485071347047209202
Training with overrides:
 {'training.batcher.size.start': 26}
✔ Auto-filled config with all values
✔ Saved config
/Users/eric/Dev/quantify-news/scripts/sent_relevance/spacy_config.cfg
You can now add your data and train your pipeline:
python -m spacy train spacy_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory:
/Users/eric/Dev/quantify-news/models/sent_relevance
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-20 13:54:32,851] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ In

2025-03-20 13:55:39,070 - git.cmd - DEBUG - Popen(['git', 'cat-file', '--batch-check'], cwd=/Users/eric/Dev/quantify-news, stdin=<valid stream>, shell=False, universal_newlines=False)
2025-03-20 13:55:39,122 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:55:39,159 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 1074
2025-03-20 13:55:39,172 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:55:39,242 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025-03-20 13:55:39,253 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1


Training time: 69.0s


2025-03-20 13:55:39,291 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025/03/20 13:55:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-20 13:55:48,856 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:55:48,910 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "GET /api/2.0/mlflow/runs/get?run_uuid=38e32c1f943c4c5dbd82ae1d8c63d5c1&run_id=38e32c1f943c4c5dbd82ae1d8c63d5c1 HTTP/1.1" 200 12950
2025-03-20 13:55:48,946 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:55:48,956 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-model HTTP/1.1" 200 2
2025-03-20 13:55:48,984 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:55:49,025 - 

🏃 View run optuna_trial_41 at: http://127.0.0.1:8080/#/experiments/485071347047209202/runs/38e32c1f943c4c5dbd82ae1d8c63d5c1
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/485071347047209202
🏃 View run opt_batch_start_size at: http://127.0.0.1:8080/#/experiments/485071347047209202/runs/06bd48db33a245e2a747dcda59201d30
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/485071347047209202


## Vector source

In [13]:
hp = "paths.vectors"
hyperparams.add(hp)

def objective(trial):
    base_model = trial.suggest_categorical(hp, ["en_core_web_sm", "en_core_web_md"])
    overrides = {hp: base_model} | best_params(hyperparams)
    return objective_base(trial, overrides)
    

In [14]:
# Note: as currently configured, the optuna_db goes into the caller of create_study ie the notebook
with mlflow.start_run(run_name="opt_base_model"):
    study.optimize(objective, n_trials=2, callbacks=[archiver])

2025-03-20 13:55:49,571 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:55:49,599 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 958
2025-03-20 13:55:49,653 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:55:49,674 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=sent_relevance_models HTTP/1.1" 200 600
2025-03-20 13:55:49,683 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:55:50,034 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/search HTTP/1.1" 200 14683


Training with overrides:
 {'paths.vectors': 'en_core_web_sm', 'training.batcher.size.start': '17'}
✔ Auto-filled config with all values
✔ Saved config
/Users/eric/Dev/quantify-news/scripts/sent_relevance/spacy_config.cfg
You can now add your data and train your pipeline:
python -m spacy train spacy_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory:
/Users/eric/Dev/quantify-news/models/sent_relevance
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-20 13:55:55,420] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       71.43    0.71

2025-03-20 13:57:05,728 - git.cmd - DEBUG - Popen(['git', 'cat-file', '--batch-check'], cwd=/Users/eric/Dev/quantify-news, stdin=<valid stream>, shell=False, universal_newlines=False)


Training time: 73.2s


2025-03-20 13:57:05,761 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:57:05,787 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 1074
2025-03-20 13:57:05,798 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:57:05,858 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025-03-20 13:57:05,867 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:57:05,885 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025/03/20 13:57:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-20 13:57:13,800 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-0

...

Training with overrides:
 {'paths.vectors': 'en_core_web_sm', 'training.batcher.size.start': '17'}
✔ Auto-filled config with all values
✔ Saved config
/Users/eric/Dev/quantify-news/scripts/sent_relevance/spacy_config.cfg
You can now add your data and train your pipeline:
python -m spacy train spacy_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory:
/Users/eric/Dev/quantify-news/models/sent_relevance
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-20 13:57:19,347] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       71.43    0.71

2025-03-20 13:58:27,431 - git.cmd - DEBUG - Popen(['git', 'cat-file', '--batch-check'], cwd=/Users/eric/Dev/quantify-news, stdin=<valid stream>, shell=False, universal_newlines=False)
2025-03-20 13:58:27,463 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1


Training time: 70.7s


2025-03-20 13:58:27,494 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 1074
2025-03-20 13:58:27,506 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:58:27,573 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025-03-20 13:58:27,582 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:58:27,606 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025/03/20 13:58:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-20 13:58:35,539 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:58:35,574 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "GET /api/2.0/mlflow/run

🏃 View run optuna_trial_43 at: http://127.0.0.1:8080/#/experiments/485071347047209202/runs/69f84d27710c4cd7a5846b00fd9622ac
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/485071347047209202
🏃 View run opt_base_model at: http://127.0.0.1:8080/#/experiments/485071347047209202/runs/9f95f7377df34fc98e6530ea4349577a
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/485071347047209202


## Bow Length


In [15]:
hp = "components.textcat_multilabel.model.length"
hyperparams.add(hp)

def objective(trial: optuna.trial.Trial):
    length = trial.suggest_int(hp, 1, 18)
    overrides = ({hp: 2**length} |
                best_params(hyperparams))
    return objective_base(trial, overrides)

In [16]:
with mlflow.start_run(run_name="opt_linear_length"):
    study.optimize(objective, n_trials=9, callbacks=[archiver])

2025-03-20 13:58:36,093 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:58:36,116 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 964
2025-03-20 13:58:36,163 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:58:36,172 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=sent_relevance_models HTTP/1.1" 200 600
2025-03-20 13:58:36,179 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:58:36,508 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/search HTTP/1.1" 200 14683


Training with overrides:
 {'components.textcat_multilabel.model.length': '262144', 'training.batcher.size.start': '17', 'paths.vectors': 'en_core_web_sm'}
✔ Auto-filled config with all values
✔ Saved config
/Users/eric/Dev/quantify-news/scripts/sent_relevance/spacy_config.cfg
You can now add your data and train your pipeline:
python -m spacy train spacy_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory:
/Users/eric/Dev/quantify-news/models/sent_relevance
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-20 13:58:41,915] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ---------

2025-03-20 13:59:51,191 - git.cmd - DEBUG - Popen(['git', 'cat-file', '--batch-check'], cwd=/Users/eric/Dev/quantify-news, stdin=<valid stream>, shell=False, universal_newlines=False)


Training time: 72.1s


2025-03-20 13:59:51,230 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:59:51,260 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 1074
2025-03-20 13:59:51,274 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:59:51,350 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025-03-20 13:59:51,360 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 13:59:51,378 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025/03/20 13:59:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-20 13:59:59,397 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-0

...

Training with overrides:
 {'components.textcat_multilabel.model.length': '262144', 'training.batcher.size.start': '17', 'paths.vectors': 'en_core_web_sm'}
✔ Auto-filled config with all values
✔ Saved config
/Users/eric/Dev/quantify-news/scripts/sent_relevance/spacy_config.cfg
You can now add your data and train your pipeline:
python -m spacy train spacy_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory:
/Users/eric/Dev/quantify-news/models/sent_relevance
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-20 14:09:11,324] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ---------

2025-03-20 14:10:05,079 - git.cmd - DEBUG - Popen(['git', 'cat-file', '--batch-check'], cwd=/Users/eric/Dev/quantify-news, stdin=<valid stream>, shell=False, universal_newlines=False)


Training time: 55.4s


2025-03-20 14:10:05,115 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:10:05,152 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 1074
2025-03-20 14:10:05,163 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:10:05,221 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025-03-20 14:10:05,228 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:10:05,250 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025/03/20 14:10:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-20 14:10:11,711 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-0

🏃 View run optuna_trial_52 at: http://127.0.0.1:8080/#/experiments/485071347047209202/runs/3d57811e15fe4bca82c23081b0a1bfb1
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/485071347047209202
🏃 View run opt_linear_length at: http://127.0.0.1:8080/#/experiments/485071347047209202/runs/e7f61d93fa0a4b56ad90f118eb6633f3
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/485071347047209202


# NGram

In [17]:
hp = "components.textcat_multilabel.model.ngram_size"
hyperparams.add(hp)

def objective(trial: optuna.trial.Trial):
    length = trial.suggest_int(hp, 1, 2)
    overrides = ({hp: length} | best_params(hyperparams))
    return objective_base(trial, overrides)

In [18]:
with mlflow.start_run(run_name="opt_ngram"):
    study.optimize(objective, n_trials=2, callbacks=[archiver])

2025-03-20 14:10:12,220 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:10:12,237 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 948
2025-03-20 14:10:12,269 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:10:12,277 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=sent_relevance_models HTTP/1.1" 200 600
2025-03-20 14:10:12,283 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:10:12,643 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/search HTTP/1.1" 200 14683


Training with overrides:
 {'components.textcat_multilabel.model.ngram_size': '1', 'training.batcher.size.start': '17', 'paths.vectors': 'en_core_web_sm', 'components.textcat_multilabel.model.length': '262144'}
✔ Auto-filled config with all values
✔ Saved config
/Users/eric/Dev/quantify-news/scripts/sent_relevance/spacy_config.cfg
You can now add your data and train your pipeline:
python -m spacy train spacy_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory:
/Users/eric/Dev/quantify-news/models/sent_relevance
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-20 14:10:16,424] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  C

2025-03-20 14:11:10,466 - git.cmd - DEBUG - Popen(['git', 'cat-file', '--batch-check'], cwd=/Users/eric/Dev/quantify-news, stdin=<valid stream>, shell=False, universal_newlines=False)
2025-03-20 14:11:10,498 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:11:10,522 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 1074
2025-03-20 14:11:10,530 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:11:10,576 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025-03-20 14:11:10,584 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:11:10,602 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025/03/20 14:11:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `inpu

🏃 View run optuna_trial_53 at: http://127.0.0.1:8080/#/experiments/485071347047209202/runs/9e284812e4974a19b50fc60175284c80
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/485071347047209202


2025-03-20 14:11:17,172 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/search HTTP/1.1" 200 14683


Training with overrides:
 {'components.textcat_multilabel.model.ngram_size': '1', 'training.batcher.size.start': '17', 'paths.vectors': 'en_core_web_sm', 'components.textcat_multilabel.model.length': '262144'}
✔ Auto-filled config with all values
✔ Saved config
/Users/eric/Dev/quantify-news/scripts/sent_relevance/spacy_config.cfg
You can now add your data and train your pipeline:
python -m spacy train spacy_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory:
/Users/eric/Dev/quantify-news/models/sent_relevance
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-03-20 14:11:20,939] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  C

2025-03-20 14:12:13,398 - git.cmd - DEBUG - Popen(['git', 'cat-file', '--batch-check'], cwd=/Users/eric/Dev/quantify-news, stdin=<valid stream>, shell=False, universal_newlines=False)
2025-03-20 14:12:13,429 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:13,452 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 1074
2025-03-20 14:12:13,461 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:13,570 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025-03-20 14:12:13,577 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:13,592 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025/03/20 14:12:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `inpu

🏃 View run optuna_trial_54 at: http://127.0.0.1:8080/#/experiments/485071347047209202/runs/88042ea6a2904452b7501fe9d25c3cb8
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/485071347047209202
🏃 View run opt_ngram at: http://127.0.0.1:8080/#/experiments/485071347047209202/runs/173b494a571a4b98a57c73b4a6234408
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/485071347047209202


# Null Model

In [19]:
class NullModel:

    def train(self, data_path: str):
        blank = spacy.blank("en")
        docs = DocBin().from_disk(data_path).get_docs(blank.vocab)
        probs = {}
        n_docs = 0
        for d in docs:
             n_docs += 1
             for c in d.cats:
                  probs[c] = probs.setdefault(c, 0) + d.cats[c]
        self.probs = {c: p / n_docs for c,p in probs.items()}
        self.n_docs = n_docs

    def eval(self, data_path):
        blank = spacy.blank("en")
        golds = list(DocBin().from_disk(data_path).get_docs(blank.vocab))
        rng = np.random.default_rng(seed=secrets.randbits(128))
        scores = pd.DataFrame.from_records([self.eval_trial(rng, golds) for _ in range(100)])
        scores = scores.select_dtypes('number')
        return scores.mean().to_dict()

    def eval_trial(self, rng, golds):
        preds = pd.concat([
            pd.Series(rng.choice(2, size=self.n_docs, p=[pc, 1-pc]), name=c)
            for c, pc in self.probs.items()
        ], axis=1)

        examples = []
        for row,gold in zip(preds.iterrows(), golds):
            d = Doc(gold.vocab).from_bytes(gold.to_bytes())
            for c in d.cats:
                d.cats[c] = row[1][c]
            examples.append(Example(d, gold))

        scorer = Scorer(default_pipeline=['textcat_multilabel'])
        return flatten_config(scorer.score_cats(examples, "cats", labels=preds.columns.to_list()))


In [20]:
null_model = NullModel()
null_model.train(train_path)
metrics = null_model.eval(dev_path)
with mlflow.start_run(run_name="null_model_expectation", nested=False) as run:
        mlflow.log_metrics(metrics)
        mlflow.log_params(null_model.probs)

2025-03-20 14:12:33,128 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:33,144 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 974
2025-03-20 14:12:33,150 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:33,168 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025-03-20 14:12:33,174 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:33,182 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/log-batch HTTP/1.1" 200 2
2025-03-20 14:12:33,188 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:33,201 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "GET /api/2.0/mlflow/runs/get?run_uuid=38d3b3773f86499fbae95e2936c35a6d&run_id=38d3b3773f86499fbae95e2936c35a6d HTTP/1.1" 

🏃 View run null_model_expectation at: http://127.0.0.1:8080/#/experiments/485071347047209202/runs/38d3b3773f86499fbae95e2936c35a6d
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/485071347047209202


# Evaluate

## Params

In [21]:
best_params(hyperparams)

2025-03-20 14:12:33,290 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:33,296 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=sent_relevance_models HTTP/1.1" 200 600
2025-03-20 14:12:33,301 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:33,710 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/search HTTP/1.1" 200 14683


{'training.batcher.size.start': '17',
 'paths.vectors': 'en_core_web_sm',
 'components.textcat_multilabel.model.ngram_size': '1',
 'components.textcat_multilabel.model.length': '262144'}

### Metrics

In [22]:
best_metrics()

2025-03-20 14:12:33,792 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:33,799 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=sent_relevance_models HTTP/1.1" 200 600
2025-03-20 14:12:33,805 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:34,178 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/search HTTP/1.1" 200 14683


{'metrics.cats_macro_p': 0.6993103448,
 'metrics.cats_micro_r': 0.7169811321,
 'metrics.cats_macro_r': 0.4926436782,
 'metrics.cats_f_per_type.WHAT.f': 0.6315789474,
 'metrics.cats_f_per_type.WHEN.f': 0.0,
 'metrics.cats_macro_f': 0.5532451819,
 'metrics.cats_f_per_type.WHEN.p': 0.0,
 'metrics.cats_f_per_type.WHO.r': 0.4,
 'metrics.cats_f_per_type.WHERE.f': 0.6666666667,
 'metrics.cats_f_per_type.WHEN.r': 0.0,
 'metrics.cats_f_per_type.WHO.p': 1.0,
 'metrics.cats_f_per_type.WHERE.p': 1.0,
 'metrics.cats_f_per_type.WHERE.r': 0.5,
 'metrics.cats_score': 0.8371653214,
 'metrics.cats_f_per_type.IRRELEVANT.f': 0.8965517241,
 'metrics.cats_f_per_type.WHAT.r': 0.6666666667,
 'metrics.cats_f_per_type.IRRELEVANT.r': 0.8965517241,
 'metrics.cats_micro_p': 0.8260869565,
 'metrics.cats_f_per_type.IRRELEVANT.p': 0.8965517241,
 'metrics.cats_micro_f': 0.7676767677,
 'metrics.cats_f_per_type.WHO.f': 0.5714285714,
 'metrics.cats_f_per_type.WHAT.p': 0.6}

### Visualize Predictions

In [23]:
apply(data_path=Path(dev_path), 
      output_file=Path("./preds.spacy"), 
      model=best_model(), 
      json_field="text", 
      batch_size=1,
      n_process=1)

2025-03-20 14:12:34,348 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:34,359 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=sent_relevance_models HTTP/1.1" 200 600
2025-03-20 14:12:34,366 - urllib3.connectionpool - DEBUG - Resetting dropped connection: 127.0.0.1
2025-03-20 14:12:34,739 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8080 "POST /api/2.0/mlflow/runs/search HTTP/1.1" 200 14683


✔ Loaded model
/Users/eric/Dev/quantify-news/mlruns/485071347047209202/88042ea6a2904452b7501fe9d25c3cb8/artifacts/optuna_trial_54/model.spacy


In [24]:
docs_pred = DocBin().from_disk("./preds.spacy")
docs_gold = DocBin().from_disk(dev_path)

In [28]:
compare = pd.DataFrame.from_records([({'text': gold.text} 
                           | {k+".x": v for k,v in gold.cats.items()} 
                           | {k+".y": 1 if v > .5 else 0 for k,v in pred.cats.items()} )
    for pred,gold in zip(docs_pred.get_docs(spacy.blank("en").vocab),
                        docs_gold.get_docs(spacy.blank("en").vocab))])
compare.sort_index(axis=1, ascending=False)

,text,WHO.y,WHO.x,WHERE.y,WHERE.x,WHEN.y,WHEN.x,WHAT.y,WHAT.x,IRRELEVANT.y,IRRELEVANT.x
0,"The other driver, a 23-year-old man, was cited...",0,1,0,0,0,0,0,1,1,0
1,"It states the boat ""shall not be operated wate...",0,0,0,0,0,0,0,0,1,1
2,champion James Holzhauer on working for the Cu...,0,0,0,0,0,0,0,0,1,1
3,#####,0,0,0,0,0,0,0,0,1,1
4,“It seems that CDOT is focused on tipping the ...,0,0,0,0,0,0,0,0,1,1
5,"The visitors apparently fell from Taft Point, ...",0,0,0,0,0,0,0,0,1,1
6,There are songs that I wish were made and I wa...,0,0,0,0,0,0,0,0,1,1
7,"Around that time, two people were shot a few b...",1,1,1,1,1,0,1,1,0,0
8,The shooter suffered a self-inflicted gunshot ...,1,1,0,0,0,0,1,1,0,0
9,[1] **,0,0,0,0,0,0,0,0,1,1


# Conclusions

At this point i'm guessing hp tuning won't help the model.
Though I haven't tried any of the more advanced model architectures yet. 
I'm not sure if this task is a) trainable or b) useful in the contest of the pipeline.
It would be better to either label more data and try again,
or move onto the next part. 